### Este notebook possui as seguintes finalidades

1- Consultar os arquivos da pasta Staging no Google Drive.
2 - Pegar todos os arquivos e transformar em um DF.
3 - Retornar os cabeçalhos.
4 - Gerar CSV final.
5 - Armazenar os arquivos .dat na pasta 30m

In [62]:
# Trabalhar com DF
import pandas as pd
# Acessar sistema operacional
import os
# Irá retornar uma lista de arquivos
import glob
# Manipulação de arquivos e diretórios
import shutil
# Identificar o separador do arquivo
import csv

##### Todos os caminhos das pastas

In [63]:
# Onde esses arquivos serão armazenados
pasta_Staging = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging'
# Arquivos de 30 minutos
pasta_Gdrive_30m = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\T1_meteo_Media_30m_2023\Ascii'
# Caminho csv 1m
arquivo_csv = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\T1_MeteoMedia_30m_2023.csv'

##### Arquivos de 30 minutos

In [64]:
# Obter a lista de arquivos .dat na pasta 1 minuto

arquivos_staging = glob.glob(f'{pasta_Staging}\\*.dat')

# Se não houver nenhum arquivo
if not arquivos_staging:
    print('Erro: a pasta está vazia.')
    # Terminar o programa.
    exit() 

In [65]:
# Há algumas linhas com erro no CSV, irei pula-las e relatar 
linhas_problematicas = []

try:
    df = pd.read_csv(arquivo_csv,sep=",",error_bad_lines=False)
except pd.errors.ParserErros as e:
    linhas_com_erros = [int(line.split()[0]) for line in str(e).split('\n')[1:-1]]
    
    # Adicione os índices das linhas problemáticas à lista linhas_problematicas
    linhas_problematicas.extend(linhas_com_erros)
    
print("Índices das linhas problemáticas:", linhas_problematicas)

Índices das linhas problemáticas: []


C:\Users\BlueShift\AppData\Local\Temp\ipykernel_55088\1069502122.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(arquivo_csv,sep=",",error_bad_lines=False)
Skipping line 613: expected 1 fields, saw 2
Skipping line 664: expected 1 fields, saw 2



##### Teste para verificar se todas as colunas estão presentes nos arquivos

In [66]:
# Criação da função que aceita os dois parâmetros
def test_all_col_exists(df_1,cols):
    # Compreensão de lista que percorre cada elemento da lista
    missing_cols = [col for col in cols if col not in df_1.columns]
    
    # O print irá trazer as colunas que não estão presentes no DF
    if missing_cols:
        print(f'A seguintes colunas não foram encontradas no {df_1}: {missing_cols}')
    else:
        print('Todas as colunas foram encontradas no DF')
        
# Colunas para verificação 
colunas_verificadas = [ 'TIMESTAMP', 'RECORD', 'WindSpeed_1', 'WindDir_1', 'WindSpeed_2', 'WindDir_2',
    'WindSpeed_3', 'WindDir_3', 'WindSpeed_4', 'WindDir_4', 'temp_1_Avg', 'temp_2_Avg', 'temp_3_Avg', 'temp_4_Avg',
    'humid_1_Avg', 'humid_2_Avg', 'humid_3_Avg', 'humid_4_Avg', 'par1_in_Avg', 'par2_in_Avg', 'par3_in_Avg',
    'par4_in_Avg', 'chuva_Tot', 'SBTempC_1_Avg', 'TargTempC_1_Avg', 'SBTempC_2_Avg', 'TargTempC_2_Avg',
    'SBTempC_3_Avg', 'TargTempC_3_Avg', 'SBTempC_4_Avg', 'TargTempC_4_Avg', 'pressao_Avg', 'rad_total_Avg',
    'rad_diff_Avg']

In [67]:
# Variável para armazenar parte do nome do arquivo
meteo_30m = "meteoMedia30"
# Lista dos arquivos verificados para ser utilizada posteriormente
arquivos_verificados = []

for arquivo in arquivos_staging:
    # Retornando apenas o nome do arquivo sem o caminho da pasta
    nome_arquivo = os.path.basename(arquivo)
    
    if meteo_30m in nome_arquivo:
        df_1 = pd.read_csv(arquivo, sep=",",skiprows = [0,2,3]) #header=1)
        # Chama a função para verificar a presença das colunas
        test_all_col_exists(df_1,colunas_verificadas)
        arquivos_verificados.append(nome_arquivo)
print(f'{len(arquivos_verificados)} arquivos foram verificados')

Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas

In [68]:
# Consultar a quantidade de linhas do arquivo csv
print(df.shape[0])

11304


In [69]:
def detectar_separador(arquivo):
    with open(arquivo,'r') as file:
        # Leitura dos primeiros 1024 bytes para detectar o separador
        conteudo = file.read(1024)
        if ';' in conteudo:
            return ';'
        elif ',' in conteudo:
            return ','
        else:
            # Use '\t' como separador padrão se nenhum for detectado
            return '\t'

##### Gerar o arquivo csv sem duplicatas

In [71]:
soma_linhas = 0
nome_csv = R'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\Teste_30m.csv'

# Crie um DataFrame vazio para iniciar
df_final = pd.DataFrame()

for arquivo in arquivos_staging:
    #separador = detectar_separador(arquivos_verificados)
    nome_arquivo = os.path.basename(arquivo)
    if nome_arquivo in arquivos_verificados:
        
        # Ler o arquivo usando o separador detectado
        df_1 = pd.read_csv(arquivo,sep=detectar_separador(arquivo), skiprows=[0, 2, 3])
        # Atualize o contador de linhas
        soma_linhas +=df_1.shape[0]
        # Concatene o DataFrame lido ao df_final existente para cada iteração
        df_final = pd.concat([df_final,df_1],ignore_index = True)

# Gerando o df_final sem duplicatas
df_sem_duplicatas = df_final.drop_duplicates()

# Ordenando o DF pela coluna Record
df_sem_duplicatas = df_sem_duplicatas.sort_values(by='RECORD')
# Resetar o indice
df_sem_duplicatas = df_sem_duplicatas.reset_index(drop=True)

df_sem_duplicatas.head()
# Use o método to_csv para salvar o DataFrame em um arquivo CSV
df_sem_duplicatas.to_csv(nome_csv,sep=',',index=False)

##### Realizar a leitura do arquivo CSV gerado, passo não necessário

In [72]:
"""nome_arquivo = "Teste.csv"

with open(nome_arquivo, 'w', newline='') as csvfile:
    # Especifique o delimitador de células (pode ser tabulação, ponto e vírgula, etc.)
    delimitador = '\t'  # Neste exemplo, usamos tabulação como delimitador
    
    # Crie um escritor CSV usando o delimitador especificado
    csvwriter = csv.writer(csvfile, delimiter=delimitador)
    
    # Escreva os dados do DataFrame no arquivo CSV
    for _, linha in df_sem_duplicatas.iterrows():
        # Converte cada valor da linha em uma lista e escreve no arquivo
        csvwriter.writerow(linha.tolist())
"""

'nome_arquivo = "Teste.csv"\n\nwith open(nome_arquivo, \'w\', newline=\'\') as csvfile:\n    # Especifique o delimitador de células (pode ser tabulação, ponto e vírgula, etc.)\n    delimitador = \'\t\'  # Neste exemplo, usamos tabulação como delimitador\n    \n    # Crie um escritor CSV usando o delimitador especificado\n    csvwriter = csv.writer(csvfile, delimiter=delimitador)\n    \n    # Escreva os dados do DataFrame no arquivo CSV\n    for _, linha in df_sem_duplicatas.iterrows():\n        # Converte cada valor da linha em uma lista e escreve no arquivo\n        csvwriter.writerow(linha.tolist())\n'

In [73]:
print(df_sem_duplicatas.shape[0])
print(soma_linhas)

1725
1725


##### Isso irá automatizar meu processo de incrementar o arquivo gerado, mas antes preciso gerar o arquivo csv final separado por virgulas

In [74]:
"""
# Concatenando ao longo das linhas Axis=0
df_concat = pd.concat([df,df_sem_duplicatas],axis=0)
df_to_csv = df_concat.sort_values(by="RECORD") 
df_to_csv = df_to_csv.reset_index(drop=True)
df_to_csv.to_csv(nome_csv, index=False)
print(f'Arquivo {nome_csv} gerado com sucesso.')
"""

'\n# Concatenando ao longo das linhas Axis=0\ndf_concat = pd.concat([df,df_sem_duplicatas],axis=0)\ndf_to_csv = df_concat.sort_values(by="RECORD") \ndf_to_csv = df_to_csv.reset_index(drop=True)\ndf_to_csv.to_csv(nome_csv, index=False)\nprint(f\'Arquivo {nome_csv} gerado com sucesso.\')\n'

In [75]:
print(df_to_csv.shape[0])

NameError: name 'df_to_csv' is not defined

#### Enviar os arquivos verificados para a pasta T1_30m

In [78]:
# Lista que armazena a quantidade de itens enviados
qntd_enviada = []
qntd_excluida = []

for arquivo in arquivos_staging:
    nome_arquivo = os.path.basename(arquivo)
    # Buscar o caminho desse arquivo na staging
    caminho_arquivo_staging = f'{pasta_Staging}\\{nome_arquivo}'
    # Buscar a pasta Staging no GDrive
    caminho_arquivo_gdrive = f'{pasta_Gdrive_30m}'
    # Copiar o arquivo de uma pasta para outra
    shutil.copy(caminho_arquivo_staging, caminho_arquivo_gdrive)
    # Para cada arquivo 1 mensagem
    print(f'Arquivo {nome_arquivo} copiado para o Gdrive')
    # Adicionar +1 para a lista
    qntd_enviada.append(+1)
    # Código abaixo é para deletar os arquivos que foram transferidos para a outra pasta
    if os.path.exists(caminho_arquivo_staging):
        #os.remove é usado para excluir só o arquivo, não as pasta
        os.remove(caminho_arquivo_staging)
        print(f'Arquivo {caminho_arquivo_staging} excluído com sucesso')
        qntd_excluida.append(+1)
    else:
        print(f'O arquivo{caminho_arquivo_staging} não existe')
print(len(qntd_enviada))
print(len(qntd_excluida))

Arquivo AmazonFACE_CR6_T1_meteoMedia30_2023_08_31_0000.dat copiado para o Gdrive
Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\AmazonFACE_CR6_T1_meteoMedia30_2023_08_31_0000.dat excluído com sucesso
Arquivo AmazonFACE_CR6_T1_meteoMedia30_2023_09_01_0000.dat copiado para o Gdrive
Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\AmazonFACE_CR6_T1_meteoMedia30_2023_09_01_0000.dat excluído com sucesso
Arquivo AmazonFACE_CR6_T1_meteoMedia30_2023_09_02_0000.dat copiado para o Gdrive
Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\AmazonFACE_CR6_T1_meteoMedia30_2023_09_02_0000.dat excluído com sucesso
Arquivo AmazonFACE_CR6_T1_meteoMedia30_2023_09_03_0000.dat copiado para o Gdrive
Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\AmazonFACE_CR6_T1_meteoMedia30_2023_09_03_0000.dat excluído com sucesso
Arquivo AmazonFACE_CR6_T1_meteoMedia30_2023_09_04_0000.dat copiado para o Gd

In [ ]:
##### Neste passo to fazendo o teste para gerar os csv juntos

In [97]:
nome_csv1 = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\T1_MeteoMedia_30m_2023.csv'
df = pd.read_csv(nome_csv1,sep=';',error_bad_lines=False)

C:\Users\BlueShift\AppData\Local\Temp\ipykernel_55088\1113510778.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(nome_csv1,sep=';',error_bad_lines=False)


In [98]:
df.head()

,TIMESTAMP,RECORD,WindSpeed_1,WindDir_1,WindSpeed_2,WindDir_2,WindSpeed_3,WindDir_3,WindSpeed_4,WindDir_4,...,TargTempC_1_Avg,SBTempC_2_Avg,TargTempC_2_Avg,SBTempC_3_Avg,TargTempC_3_Avg,SBTempC_4_Avg,TargTempC_4_Avg,pressao_Avg,rad_total_Avg,rad_diff_Avg
0,01/01/2023 00:00,4473,1.252.222,339.979,0,0,0,0,0,0,...,2.236.845,2.236.646,1.834.034,2.240.627,2.006.042,2.233.097,1.977.277,9.962.813,0,0
1,01/01/2023 00:30,4474,0.8597224,7.502.235,0,0,0,0,0,0,...,2.223.256,2.225.941,1.821.121,2.227.962,1.994.649,2.221.755,19.657,9.957.971,0,0
2,01/01/2023 01:00,4475,1.006.111,8.029.311,0,0,0,0,0,0,...,2.223.549,2.219.495,1.820.764,2.220.754,1.991.658,2.215.539,1.964.685,9.951.842,0,0
3,01/01/2023 01:30,4476,1.181.666,7.483.522,0,0,0,0,0,0,...,2.208.759,2.209.636,1.808.706,2.211.353,1.980.769,2.206.184,1.952.285,9.945.773,0,0
4,01/01/2023 02:00,4477,0.9066666,8.430.209,0,0,0,0,0,0,...,2.203.237,2.200.943,1.802.293,2.202.038,197.412,2.196.756,194.692,9.942.584,0,0


In [100]:
nome_csv2 = R'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\Teste_30m1.csv'
df.to_csv(nome_csv2,sep=',',index=False)

In [101]:
# Concatenando ao longo das linhas Axis=0
df_concat = pd.concat([df,df_sem_duplicatas],axis=0)
df_to_csv = df_concat.sort_values(by="RECORD") 
df_to_csv = df_to_csv.reset_index(drop=True)
df_to_csv.to_csv(nome_csv2, index=False)
print(f'Arquivo {nome_csv} gerado com sucesso.')

Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\Teste_30m.csv gerado com sucesso.
